# Chapter 16: Natural Language Processing with RNNs and Attention

Bab ini mengeksplorasi teknik pemrosesan bahasa alami (NLP) yang lebih maju. Kita akan beralih dari sekadar memproses urutan angka sederhana ke memproses teks, di mana konteks dan hubungan antar kata sangat krusial.

## Tujuan Pembelajaran:
1. **Membangun Char-RNN**: Menghasilkan teks secara kreatif.
2. **Memahami Sentiment Analysis**: Klasifikasi teks menggunakan Embedding.
3. **Eksplorasi Encoder-Decoder**: Dasar dari sistem terjemahan mesin.
4. **Mekanisme Attention**: Fokus pada bagian data yang relevan.
5. **Transformer**: Arsitektur modern yang mendasari model seperti GPT dan BERT.

## 1. Menghasilkan Teks Karakter demi Karakter (Char-RNN)

Char-RNN dilatih untuk memprediksi karakter berikutnya dalam urutan teks. Ini memungkinkan model untuk mempelajari struktur bahasa dari tingkat paling dasar (huruf, spasi, tanda baca).

### Proses Data:
- **Tokenisasi**: Mengubah karakter menjadi ID numerik.
- **Sliding Window**: Memotong teks panjang menjadi urutan kecil untuk latihan.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# 1. Tokenisasi tingkat karakter
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
shakespeare_text = "To be, or not to be, that is the question."
tokenizer.fit_on_texts([shakespeare_text])

max_id = len(tokenizer.word_index) # Jumlah karakter unik
dataset_size = len(tokenizer.texts_to_sequences([shakespeare_text])[0])

print(f"Jumlah karakter unik: {max_id}")
print(f"Total panjang teks: {dataset_size}")

# 2. Membangun model GRU untuk Char-RNN
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
print("Model Char-RNN siap dilatih.")

## 2. Stateful vs Stateless RNN

| Fitur | Stateless RNN | Stateful RNN |
|---|---|---|
| **Memory** | Reset di setiap batch | Dipertahankan antar batch |
| **Urutan Data** | Boleh diacak | Harus berurutan |
| **Konteks** | Jangka pendek | Jangka sangat panjang |

Stateful RNN sangat berguna jika kita ingin model mengingat konteks dari ribuan langkah waktu sebelumnya, namun memerlukan manajemen batch yang sangat disiplin.

## 3. Sentiment Analysis (Analisis Sentimen)

Dalam analisis sentimen, kita mengubah kata menjadi vektor padat menggunakan lapisan **Embedding**. Kita juga menggunakan **Masking** agar model mengabaikan padding (token kosong) yang digunakan untuk menyamakan panjang kalimat.

In [ ]:
# Model Analisis Sentimen untuk dataset IMDb
model_sentiment = keras.models.Sequential([
    # input_dim: ukuran kosakata, output_dim: dimensi vektor embedding
    keras.layers.Embedding(input_dim=10000, output_dim=128, mask_zero=True, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model_sentiment.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print("Model Sentiment Analysis berhasil dikonfigurasi dengan Masking.")

## 4. Encoder-Decoder untuk Terjemahan Mesin

Arsitektur ini menggunakan dua bagian utama:
1. **Encoder**: Memproses kalimat bahasa asal (misal: Inggris) menjadi satu vektor konteks.
2. **Decoder**: Mengambil vektor tersebut dan menghasilkan kalimat bahasa tujuan (misal: Indonesia) kata demi kata.

**Bottleneck**: Vektor konteks tunggal sulit menampung informasi dari kalimat yang sangat panjang.

## 5. Mekanisme Attention

Attention memecahkan masalah bottleneck dengan membiarkan Decoder "melihat" seluruh urutan output dari Encoder dan memilih bagian mana yang paling relevan untuk kata yang sedang diterjemahkan saat ini.

- **Bahdanau Attention**: Sering disebut 'Additive Attention'.
- **Luong Attention**: Sering disebut 'Multiplicative Attention'.

## 6. Transformer: Attention Is All You Need

Transformer adalah arsitektur yang membuang RNN sepenuhnya dan mengandalkan **Self-Attention**. Ini memungkinkan pelatihan yang jauh lebih cepat karena data diproses secara paralel (tidak sekuensial).

### Komponen Kunci Transformer:
- **Multi-Head Attention**: Melihat hubungan kata dari berbagai perspektif.
- **Positional Encoding**: Menambahkan informasi urutan karena Transformer tidak memproses data secara berurutan.
- **Residual Connections**: Memudahkan aliran gradien pada network yang dalam.

In [ ]:
# Contoh implementasi Multi-Head Attention dari Keras
num_heads = 8
key_dim = 64
mha_layer = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)

# Simulasi input (batch_size, sequence_length, embedding_dim)
query = tf.random.uniform((32, 50, 512))
value = query # Self-attention

output_tensor = mha_layer(query, value)
print(f"Output shape dari Multi-Head Attention: {output_tensor.shape}")

## 7. Model Bahasa Modern (GPT & BERT)

- **BERT (Bidirectional Encoder Representations from Transformers)**: Dilatih untuk memahami konteks dari dua arah (kiri ke kanan dan kanan ke kiri). Sangat bagus untuk pemahaman teks.
- **GPT (Generative Pre-trained Transformer)**: Berfokus pada generasi teks (memprediksi kata berikutnya). Dilatih hanya dari satu arah (kiri ke kanan).

## Ringkasan Bab 16
1. RNN sangat kuat untuk urutan, tetapi memiliki batas memori jangka panjang.
2. **Embedding** dan **Masking** adalah dasar dari pengolahan teks modern.
3. **Attention** merevolusi cara kita menangani urutan panjang.
4. **Transformer** saat ini menjadi standar emas dalam dunia NLP, mengungguli RNN dalam hampir semua tugas besar.